In [ ]:
import pymc3 as pm
import theano
import theano.tensor as tt
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def build_model0(
    s,
    m,
    gamma_hyper=1e-2,
    pi_hyper=1e-1,
    rho_hyper=1e-0,
    epsilon_hyper=0.01,
    alpha_hyper=1000.0,
):
    
    n, g = m.shape

    with pm.Model() as model:
        _y = pm.Data('_y', np.zeros_like(m))
        _m = pm.Data('_m', m)
        
        gamma = pm.Beta('gamma', alpha=gamma_hyper, beta=gamma_hyper, shape=(s, g))
        rho = pm.Dirichlet('rho', rho_hyper * tt.ones(s))
        pi = pm.Dirichlet('pi', pi_hyper * rho * s, shape=(n, s))
        
        epsilon = pm.Beta('epsilon', alpha=1.0, beta=1 / epsilon_hyper, shape=(n, 1))
        alpha = pm.Gamma('alpha', alpha=alpha_hyper, beta=1.0)
        
        p_noerr = pm.Deterministic("p_noerr", pm.math.dot(pi, gamma))
        p = pm.Deterministic(
            "p", (1 - epsilon / 2) * (p_noerr) + (epsilon / 2) * (1 - p_noerr)
        )

        y = pm.BetaBinomial(
            "y", alpha=alpha * p, beta=alpha * (1 - p), n=_m, observed=_y,
        )

    return model


def build_model1(
    s,
    m,
    gamma_hyper=1e-2,
    pi_hyper=1e-1,
    rho_hyper=1e-0,
    epsilon_hyper=0.01,
    alpha_hyper=1000.0,
    y=None,
):
    
    n, g = m.shape

    with pm.Model() as model:        
        gamma = pm.Beta('gamma', alpha=gamma_hyper, beta=gamma_hyper, shape=(s, g))
        rho = pm.Dirichlet('rho', rho_hyper * tt.ones(s))
        pi = pm.Dirichlet('pi', pi_hyper * rho * s, shape=(n, s))
        
        epsilon = pm.Beta('epsilon', alpha=1.0, beta=1 / epsilon_hyper, shape=(n, 1))
        alpha = pm.Gamma('alpha', alpha=alpha_hyper, beta=1.0)
        
        p_noerr = pm.Deterministic("p_noerr", pm.math.dot(pi, gamma))
        p = pm.Deterministic(
            "p", (1 - epsilon / 2) * (p_noerr) + (epsilon / 2) * (1 - p_noerr)
        )

        y = pm.BetaBinomial(
            "y", alpha=alpha * p, beta=alpha * (1 - p), n=m, observed=y,
        )

    return model
    

In [ ]:
model0 = build_model0(3, np.ones((10, 20)))
model0

In [ ]:
pm.model_to_graphviz(model0)

In [ ]:
with model0:
    prior_pred = pm.sample_prior_predictive(samples=1)

In [ ]:
plt.plot(prior_pred['rho'])

In [ ]:
sns.heatmap(prior_pred['pi'])

In [ ]:
sns.heatmap(prior_pred['gamma'])

In [ ]:
model0._y.set_value(prior_pred['y'])
with model0:
    trace = pm.sample(return_inferencedata=True)

In [ ]:
model0._y.set_value(prior_pred['y'])
with model0:
    mapest = pm.find_MAP()

In [ ]:
pm.__version__

In [ ]:
for chain_i in range(4):
    plt.plot(trace.posterior['pi'][chain_i,:,6,0], label=chain_i)
plt.legend()